# Aerosol calibration

Use the AR6 per-species ERFari calibrations, from Chapter 6 Fig. 6.12.

For piClim-HC, the marginal contributions to aerosol forcing should possibly be EESC-weighted. But they are so small in the mean that the difference by just using a radiative efficiency adjustment is probably negligible, and this effect is probably not well researched (putting in an EESC dependence will require a code modification that may not be justified). For now we will base on present-day contribution to net forcing.

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as pl
import json
import scipy.stats

from tqdm import tqdm

In [ ]:
df_emis = pd.read_csv('../data/rcmip/rcmip-emissions-annual-means-v5-1-0.csv')
df_conc = pd.read_csv('../data/rcmip/rcmip-concentrations-annual-means-v5-1-0.csv')

In [ ]:
emitted_species = [
    'Sulfur', 'BC', 'OC', 'NH3', 'NOx', 'VOC', 'CO',
]

In [ ]:
concentration_species = [
    'CH4', 'N2O',
    'CFC-11', 'CFC-12', 'CFC-113', 'CFC-114', 'CFC-115',
    'HCFC-22', 'HCFC-141b', 'HCFC-142b',
    'CCl4', 'CHCl3', 'CH2Cl2', 'CH3Cl', 'CH3CCl3', 'CH3Br',
    'Halon-1211', 'Halon-1301', 'Halon-2402',
]

In [ ]:
species_out = {}
for ispec, species in enumerate(emitted_species):
    species_rcmip_name = species.replace("-", "")
    emis_in = df_emis.loc[
        (df_emis['Scenario']=='ssp245') & (df_emis['Variable'].str.endswith("|"+species_rcmip_name)) & 
        (df_emis['Region']=='World'), '1750':'2100'
    ].interpolate(axis=1).values.squeeze()
    species_out[species] = emis_in
    
for ispec, species in enumerate(concentration_species):
    species_rcmip_name = species.replace("-", "")
    conc_in = df_conc.loc[
        (df_conc['Scenario']=='ssp245') & (df_conc['Variable'].str.endswith("|"+species_rcmip_name)) & 
        (df_conc['Region']=='World'), '1750':'2100'
    ].interpolate(axis=1).values.squeeze()
    species_out[species] = conc_in

In [ ]:
species_df = pd.DataFrame(species_out, index=range(1750,2101))
species_df

In [ ]:
species_df.loc[2019,'CFC-11':]

In [ ]:
radiative_efficiency = pd.Series({
    'CFC-11': 0.25941,
    'CFC-12': 0.31998,
    'CFC-113': 0.30142,
    'CFC-114': 0.31433,
    'CFC-115': 0.24625,
    'HCFC-22': 0.21385,
    'HCFC-141b': 0.16065,
    'HCFC-142b': 0.19329,
    'CCl4': 0.16616,
    'CHCl3': 0.07357,
    'CH2Cl2': 0.02882,
    'CH3Cl': 0.00466,
    'CH3CCl3': 0.06454,
    'CH3Br': 0.00432,
    'Halon-1211': 0.30014,
    'Halon-1301': 0.29943,
    'Halon-2402': 0.31169,
})

In [ ]:
hc_erf = (species_df.loc[2019,'CFC-11':] * radiative_efficiency/1000)
hc_sum = hc_erf.sum()

In [ ]:
erfari_emitted = pd.Series({
    'Sulfur': -0.234228,
    'BC': 0.144702,
    'OC': -0.072143,
    'NH3': -0.033769,
    'NOx': -0.009166,
    'VOC': -0.002573,
    'CO': 0,
    'CH4': -0.002653,
    'N2O': -0.00209,
    'CFC-11': -0.00808 * hc_erf['CFC-11']/hc_sum,
    'CFC-12': -0.00808 * hc_erf['CFC-12']/hc_sum,
    'CFC-113': -0.00808 * hc_erf['CFC-113']/hc_sum,
    'CFC-114': -0.00808 * hc_erf['CFC-114']/hc_sum,
    'CFC-115': -0.00808 * hc_erf['CFC-115']/hc_sum,
    'HCFC-22': -0.00808 * hc_erf['HCFC-22']/hc_sum,
    'HCFC-141b': -0.00808 * hc_erf['HCFC-141b']/hc_sum,
    'HCFC-142b': -0.00808 * hc_erf['HCFC-142b']/hc_sum,
    'CCl4': -0.00808 * hc_erf['CCl4']/hc_sum,
    'CHCl3': -0.00808 * hc_erf['CHCl3']/hc_sum,
    'CH2Cl2': -0.00808 * hc_erf['CH2Cl2']/hc_sum,
    'CH3Cl': -0.00808 * hc_erf['CH3Cl']/hc_sum,
    'CH3CCl3': -0.00808 * hc_erf['CH3CCl3']/hc_sum,
    'CH3Br': -0.00808 * hc_erf['CH3Br']/hc_sum,
    'Halon-1211': -0.00808 * hc_erf['Halon-1211']/hc_sum,
    'Halon-1301': -0.00808 * hc_erf['Halon-1301']/hc_sum,
    'Halon-2402': -0.00808 * hc_erf['Halon-2402']/hc_sum,
})
erfari_emitted

In [ ]:
# erfari radiative efficiency per Mt or ppb or ppt
re = erfari_emitted / (species_df.loc[2019, :] - species_df.loc[1750, :])

In [ ]:
re

In [ ]:
samples = 1500000

In [ ]:
scalings = scipy.stats.uniform.rvs(np.minimum(re*2, 0), np.maximum(re*2, 0)-np.minimum(re*2, 0), size=(samples, 26), random_state=3729329)

In [ ]:
#pl.hist(scalings[:,1])

In [ ]:
erfari = np.zeros((351, samples))

for i in tqdm(range(samples)):
    erfari[:, i] = ((species_df - species_df.loc[1750])*scalings[i,:]).sum(axis=1)

In [ ]:
#np.sum(np.isnan(erfari))

In [ ]:
pl.hist(erfari[255:265, :].mean(axis=0))
np.percentile(erfari[255:265, :].mean(axis=0), (5,50,95))
# # target -0.71 -0.58 ? -0.23 -0.14 for 16-84
# np.percentile(erfari[255:265, :].mean(axis=0), (5, 16, 50, 84, 95))

In [ ]:
df = pd.DataFrame(scalings, columns=[
        'Sulfur',
        'BC',
        'OC',
        'NH3',
        'NOx',
        'VOC',
        'CO',
        'CH4',
        'N2O',
        'CFC-11',
        'CFC-12',
        'CFC-113',
        'CFC-114',
        'CFC-115',
        'HCFC-22',
        'HCFC-141b',
        'HCFC-142b',
        'CCl4',
        'CHCl3',
        'CH2Cl2',
        'CH3Cl',
        'CH3CCl3',
        'CH3Br',
        'Halon-1211',
        'Halon-1301',
        'Halon-2402',
    ]
)

In [ ]:
df

In [ ]:
df.to_csv('../data/parameter_sets/erfari.csv', index=False)